In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_pickle("similar_pods_based_on_episodes.pkl")

In [3]:
pods1 = pd.read_pickle("pod_descriptions.pkl")
pods2 = pd.read_pickle("pods_mean_vec_description.pkl")

In [5]:
temp = pods2[["itunes_id","vector"]].merge(pods1, on = "itunes_id").merge(df, on = "itunes_id")[["itunes_id", "matches"]]

In [20]:
_ids = []
match_id = []
score = []
for i in range(temp.shape[0]):
    if (i+1)%100 == 0:
        print(i+1, end = " | ")
    current_matches = temp.loc[i,"matches"]
    match_id.extend(current_matches["itunes_id"].to_list())
    score.extend(current_matches["score"].to_list())
    _ids.extend([temp.loc[i,"itunes_id"]]*(current_matches.shape[0]))

100 | 200 | 300 | 400 | 500 | 600 | 700 | 800 | 900 | 1000 | 1100 | 1200 | 1300 | 1400 | 1500 | 1600 | 1700 | 1800 | 1900 | 2000 | 2100 | 2200 | 2300 | 2400 | 2500 | 2600 | 2700 | 2800 | 2900 | 3000 | 3100 | 3200 | 3300 | 3400 | 3500 | 3600 | 3700 | 3800 | 3900 | 4000 | 4100 | 4200 | 4300 | 4400 | 4500 | 4600 | 4700 | 4800 | 4900 | 5000 | 5100 | 5200 | 5300 | 5400 | 5500 | 5600 | 5700 | 5800 | 5900 | 6000 | 6100 | 6200 | 6300 | 6400 | 6500 | 6600 | 6700 | 6800 | 6900 | 7000 | 7100 | 7200 | 7300 | 7400 | 7500 | 7600 | 7700 | 7800 | 7900 | 8000 | 8100 | 8200 | 8300 | 8400 | 8500 | 8600 | 8700 | 8800 | 8900 | 9000 | 9100 | 9200 | 9300 | 9400 | 9500 | 9600 | 9700 | 9800 | 9900 | 10000 | 10100 | 10200 | 10300 | 10400 | 10500 | 10600 | 10700 | 10800 | 10900 | 11000 | 11100 | 11200 | 11300 | 11400 | 11500 | 11600 | 11700 | 11800 | 11900 | 12000 | 12100 | 12200 | 12300 | 12400 | 12500 | 12600 | 12700 | 12800 | 12900 | 13000 | 13100 | 13200 | 13300 | 13400 | 13500 | 13600 | 13700 | 13800 | 1390

In [21]:
all_matches = pd.DataFrame(columns=["itunes_id","match_id","score"])
all_matches["itunes_id"] = _ids
all_matches["match_id"] = match_id
all_matches["score"] = score

In [23]:
all_pods = pods2[["itunes_id","vector"]].merge(pods1, on = "itunes_id").merge(df, on = "itunes_id")\
    [["itunes_id", "titles", "descriptions", "genre", "subgenre", "artwork_url"]]

In [24]:
vecs = pods2[["itunes_id","vector"]].merge(pods1, on = "itunes_id").merge(df, on = "itunes_id")["vector"].to_numpy()
vecs = np.array([vec for vec in vecs])

In [25]:
sims = cosine_similarity(vecs)
top100 = np.argpartition(-sims, 101)[:,:101]
val = np.array([sims[i,:][top100[i,:]] for i in range(sims.shape[0])])
top100_idx = np.argsort(-val)
matches_meanvec = np.array([top100[i,top100_idx[i]] for i in range(sims.shape[0])])
scores_meanvec = np.array([val[i,top100_idx[i]] for i in range(sims.shape[0])])
matches_meanvec = matches_meanvec[:,1:]
scores_meanvec = scores_meanvec[:,1:]
matches_meanvec = matches_meanvec.reshape(19710*100)
scores_meanvec = scores_meanvec.reshape(19710*100)

In [104]:
ids = all_pods["itunes_id"].to_list()
id_list = [_id for _id in ids for i in range(100)]

In [131]:
all_matches_meanvec = pd.DataFrame(columns=["itunes_id","match_id","score"])
all_matches_meanvec["itunes_id"] = id_list
all_matches_meanvec["match_id"] = all_pods["itunes_id"].loc[matches_meanvec].to_list()
all_matches_meanvec["score"] = scores_meanvec

In [126]:
all_matches.to_pickle("all_matches_episode_based.pkl")
all_matches_meanvec.to_pickle("all_matches_mean_vec_based.pkl")
all_pods.to_pickle("all_pods.pkl")

In [150]:
all_pods[all_pods["titles"] == "The Liberty Yell"]

,itunes_id,titles,descriptions,genre,subgenre,artwork_url
16764,1472959248,The Liberty Yell,Welcome to The Liberty Yell! A podcast all abo...,Sports,Hockey,https://is2-ssl.mzstatic.com/image/thumb/Podca...


In [151]:
idx = 16764 #np.random.randint(all_pods.shape[0])
pod_id = all_pods.loc[idx, "itunes_id"]
all_pods.iloc[idx:idx+1]

,itunes_id,titles,descriptions,genre,subgenre,artwork_url
16764,1472959248,The Liberty Yell,Welcome to The Liberty Yell! A podcast all abo...,Sports,Hockey,https://is2-ssl.mzstatic.com/image/thumb/Podca...


In [152]:
results = all_matches[all_matches["itunes_id"] == pod_id][["match_id", "score"]]
results.columns = ["itunes_id", "score"]
results.merge(all_pods)[:20]

,itunes_id,score,titles,descriptions,genre,subgenre,artwork_url
0,1524083342,1.900000,Real Kyper at Noon,Welcome to a new NHL show from Nick Kypreos an...,Sports,Hockey,https://is1-ssl.mzstatic.com/image/thumb/Podca...
1,699005995,1.046823,Pucks On Net,A Vancouver hockey podcast that doesn't talk a...,Sports,Hockey,https://is2-ssl.mzstatic.com/image/thumb/Podca...
2,1525301754,0.878788,NBA Gambling Podcast,A daily basketball podcast covering NBA bettin...,Sports,Basketball,https://is1-ssl.mzstatic.com/image/thumb/Podca...
3,1418357960,0.617021,The Isles Faithful Podcast,The Isles Faithful Podcast An NHL Podcast on T...,Sports,Hockey,https://is4-ssl.mzstatic.com/image/thumb/Podca...
4,1497643797,0.583333,Avs Offside By A Mile,Offside By A Mile is your home of the Colorado...,Sports,Hockey,https://is3-ssl.mzstatic.com/image/thumb/Podca...
5,1512378723,0.555556,The Legends of League Podcast,The biggest names in Rugby League talking all ...,Sports,Rugby,https://is2-ssl.mzstatic.com/image/thumb/Podca...
6,1521067784,0.500000,The Yellowstone Podcast with Stef and Sheila,Stef and Sheila dig into each new Yellowstone ...,TV & Film,TV Reviews,https://is5-ssl.mzstatic.com/image/thumb/Podca...
7,1456631897,0.482759,Otaku Summit,Three friends discuss all things anime droppin...,Leisure,Animation & Manga,https://is1-ssl.mzstatic.com/image/thumb/Podca...
8,1013135614,0.373333,RotoUnderworld Radio - Fantasy Football Show,Fantasy football podcast providing analysis fr...,Sports,Sports,https://is4-ssl.mzstatic.com/image/thumb/Podca...
9,1509390785,0.368421,The Yard - A Los Angeles Dodgers Fan Podcast,We're here to talk about all things Dodgers! F...,Sports,Baseball,https://is4-ssl.mzstatic.com/image/thumb/Podca...


In [153]:
results = all_matches_meanvec[all_matches_meanvec["itunes_id"] == pod_id][["match_id", "score"]]
results.columns = ["itunes_id", "score"]
results.merge(all_pods)[:20]

,itunes_id,score,titles,descriptions,genre,subgenre,artwork_url
0,1419742734,0.991841,Dropping the Gloves,"Hey Everyone! I'm John Scott, a retired NHL Al...",Sports,Hockey,https://is5-ssl.mzstatic.com/image/thumb/Podca...
1,1459001246,0.988822,The Vegas Chirp,This podcast is a Vegas Golden Knights Podcast...,Sports,Hockey,https://is5-ssl.mzstatic.com/image/thumb/Podca...
2,1054081827,0.988522,Game Theory Podcast,"Sam Vecenie, a Senior NBA Writer at The Athlet...",Sports,Basketball,https://is3-ssl.mzstatic.com/image/thumb/Podca...
3,1444931422,0.988251,Canucks Conversation,Vancouver Canucks hockey podcast by Faber and ...,Sports,Hockey,https://is4-ssl.mzstatic.com/image/thumb/Podca...
4,1066326306,0.988232,Blueshirts Breakaway: A Show about the New Yor...,Ryan Mead and Greg Kaplan host this weekly New...,Sports,Hockey,https://is3-ssl.mzstatic.com/image/thumb/Podca...
5,699005995,0.988127,Pucks On Net,A Vancouver hockey podcast that doesn't talk a...,Sports,Hockey,https://is2-ssl.mzstatic.com/image/thumb/Podca...
6,1474726168,0.987349,The Boys of 161st Street - Yankees MLB Podcast,"Yankees Podcast, we like to have fun. We hope ...",Sports,Baseball,https://is5-ssl.mzstatic.com/image/thumb/Podca...
7,1457738054,0.987082,George's Box - Yankees MLB Podcast,George's Box is a baseball podcast hosted by J...,Sports,Baseball,https://is5-ssl.mzstatic.com/image/thumb/Podca...
8,976803232,0.986851,Section 10 Podcast,This is the best Red Sox podcast in the world....,Sports,Baseball,https://is3-ssl.mzstatic.com/image/thumb/Podca...
9,1526239005,0.986522,FlipSide 4 Your Ears,FlipSide 4 Your Ears Podcast give you a differ...,News,Sports News,https://is5-ssl.mzstatic.com/image/thumb/Podca...


In [138]:
all_meanvecs = pods2[["itunes_id","vector"]].merge(pods1, on = "itunes_id").merge(df, on = "itunes_id")[["itunes_id","vector"]]

In [139]:
all_meanvecs.to_pickle("all_mean_vectors")

In [154]:
all_matches_meanvec[all_matches_meanvec["itunes_id"] == pod_id]

,itunes_id,match_id,score
1676400,1472959248,1419742734,0.991841
1676401,1472959248,1459001246,0.988822
1676402,1472959248,1054081827,0.988522
1676403,1472959248,1444931422,0.988251
1676404,1472959248,1066326306,0.988232
...,...,...,...
1676495,1472959248,1279724711,0.982419
1676496,1472959248,1501466398,0.982397
1676497,1472959248,1441917610,0.982394
1676498,1472959248,1440512452,0.982386


In [155]:
all_pods[all_pods["itunes_id"] == 1419742734]

,itunes_id,titles,descriptions,genre,subgenre,artwork_url
16737,1419742734,Dropping the Gloves,"Hey Everyone! I'm John Scott, a retired NHL Al...",Sports,Hockey,https://is5-ssl.mzstatic.com/image/thumb/Podca...
